In [1]:
import sys
sys.path.append('../')

import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

In [2]:
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_756_hour,rides_previous_755_hour,rides_previous_754_hour,rides_previous_753_hour,rides_previous_752_hour,rides_previous_751_hour,rides_previous_750_hour,rides_previous_749_hour,rides_previous_748_hour,rides_previous_747_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,0.0,0.0,5.0,4.0,3.0,4.0,3.0,2022-02-01 12:00:00,4,7.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,2.0,12.0,12.0,4.0,2.0,7.0,2022-02-02 12:00:00,4,5.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,0.0,0.0,11.0,13.0,10.0,5.0,5.0,2022-02-03 12:00:00,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,0.0,2.0,3.0,16.0,5.0,6.0,5.0,2022-02-04 12:00:00,4,4.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,1.0,3.0,0.0,2.0,5.0,4.0,12.0,2022-02-05 12:00:00,4,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27 12:00:00,199,0.0
87504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28 12:00:00,199,0.0
87505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29 12:00:00,199,0.0
87506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30 12:00:00,199,0.0


In [3]:
from datetime import datetime
from src.data_split import train_test_split

In [4]:
X_train, y_train, X_test, y_test = train_test_split(df, 
                                                    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
                                                    target_column_name='target_rides_next_hour')
print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(31440, 758)
y_train.shape=(31440,)
X_test.shape=(56068, 758)
y_test.shape=(56068,)


In [5]:
# Create a Baseline model class

import numpy as np

class BaselineModelPreviousHour:
    """
    Prediction = actual demand observed in the last hour
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_1_hour']
    

In [6]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)
predictions

0        2.0
1        2.0
2        7.0
3        3.0
4        2.0
        ... 
56063    0.0
56064    0.0
56065    0.0
56066    0.0
56067    0.0
Name: rides_previous_1_hour, Length: 56068, dtype: float32

In [7]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae:.4f}')

3.6850


## Baseline model - 2 
we saw our time series data has a weekly seasonality, which means in order to predict,  for example what's going to happen in the next hour and the `next hour is Friday at 5:00` for example. We can look back at `what happened exactly last friday at 5:00` and use that historical demand as our estimate.

In [8]:
import pandas as pd
import numpy as np

class BaselineModelPreviousWeek:
    """
    Prediction = actual demand observed at t - 7 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_{24 * 7}_hour']

In [9]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)
predictions

0        2.0
1        1.0
2        5.0
3        5.0
4        5.0
        ... 
56063    0.0
56064    0.0
56065    0.0
56066    0.0
56067    0.0
Name: rides_previous_168_hour, Length: 56068, dtype: float32

In [10]:
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae:.4f}')

4.6049


## Baseline model - 3
In this model we are going to average of the error values observed 7 days, 14 days, 21 days, and 28 days ago i.e. we again leverage the weekly seasonality in the data. Averaging estimates is a strategy that usually results in improvements 

In [11]:
class BaselineModelLast4Weeks:
    """
    Prediction = actual demand observed at t - 7 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return 0.25 * (
            X_test[f'rides_previous_{24 * 7}_hour'] + 
            X_test[f'rides_previous_{24 * 7 * 2}_hour'] + 
            X_test[f'rides_previous_{24 * 7 * 3}_hour'] + 
            X_test[f'rides_previous_{24 * 7 * 4}_hour'])

In [12]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae:.4f}')

3.9414


For tabular data the `best algorithms` are supposed to be boosting `algorithms like XGBoost, LightGBM, CatBoost` etc. Mostly if any of these 3 doesn't work the problem is with the data.